## Question 1

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime 
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
import xgboost as xgb
from xgboost import plot_importance

In [2]:
df = pd.read_csv("train.csv")
df.head()

,gvkey,datadate,conm,at,cogs,csho,dvp,ebit,gp,ib,...,xido,xopr,mkvalt,sic,bankrupt,Altman_X1,Altman_X2,Altman_X3,Altman_X4,Altman_X5
0,1240,2002-01-31,ALBERTSON'S INC,15967.000,26094.000,407.000,0.0,1787.000,11837.000,501.000,...,0.000,35118.000,11701.2500,5411.0,0,0.064320,0.339074,0.111918,1.164072,2.375587
1,1655,2002-01-31,ANGELICA CORP,290.865,246.466,8.608,0.0,12.856,103.597,1.629,...,-24.338,326.107,101.5744,7200.0,0,0.161449,0.488845,0.044199,0.680060,1.203524
2,1864,2002-01-31,REX AMERICAN RESOURCES CORP,307.329,325.912,8.163,0.0,22.236,138.591,22.554,...,-0.245,438.069,240.4004,2860.0,0,0.300186,0.438319,0.072352,1.508575,1.511419
3,1878,2002-01-31,AUTODESK INC,902.444,124.903,55.644,0.0,131.804,822.588,90.313,...,0.000,768.484,2281.4040,7370.0,0,0.214106,0.079664,0.146052,6.114234,1.049917
4,2436,2002-01-31,CALERES INC,700.898,1044.110,17.484,0.0,54.842,711.738,0.949,...,-4.911,1674.299,278.6950,3140.0,0,0.321669,0.206953,0.078245,0.627350,2.505141


In [3]:
to_calculate =  df[['Altman_X1','Altman_X2','Altman_X3','Altman_X4','Altman_X5']]
to_calculate.head()

,Altman_X1,Altman_X2,Altman_X3,Altman_X4,Altman_X5
0,0.064320,0.339074,0.111918,1.164072,2.375587
1,0.161449,0.488845,0.044199,0.680060,1.203524
2,0.300186,0.438319,0.072352,1.508575,1.511419
3,0.214106,0.079664,0.146052,6.114234,1.049917
4,0.321669,0.206953,0.078245,0.627350,2.505141


In [4]:
df['Z_Score'] = to_calculate.sum(axis = 1)
df.head()

,gvkey,datadate,conm,at,cogs,csho,dvp,ebit,gp,ib,...,xopr,mkvalt,sic,bankrupt,Altman_X1,Altman_X2,Altman_X3,Altman_X4,Altman_X5,Z_Score
0,1240,2002-01-31,ALBERTSON'S INC,15967.000,26094.000,407.000,0.0,1787.000,11837.000,501.000,...,35118.000,11701.2500,5411.0,0,0.064320,0.339074,0.111918,1.164072,2.375587,4.054972
1,1655,2002-01-31,ANGELICA CORP,290.865,246.466,8.608,0.0,12.856,103.597,1.629,...,326.107,101.5744,7200.0,0,0.161449,0.488845,0.044199,0.680060,1.203524,2.578078
2,1864,2002-01-31,REX AMERICAN RESOURCES CORP,307.329,325.912,8.163,0.0,22.236,138.591,22.554,...,438.069,240.4004,2860.0,0,0.300186,0.438319,0.072352,1.508575,1.511419,3.830851
3,1878,2002-01-31,AUTODESK INC,902.444,124.903,55.644,0.0,131.804,822.588,90.313,...,768.484,2281.4040,7370.0,0,0.214106,0.079664,0.146052,6.114234,1.049917,7.603973
4,2436,2002-01-31,CALERES INC,700.898,1044.110,17.484,0.0,54.842,711.738,0.949,...,1674.299,278.6950,3140.0,0,0.321669,0.206953,0.078245,0.627350,2.505141,3.739357


In [5]:
labels = []
for row in df['Z_Score']:
    if row >= 2.675:
        labels.append(0)
    else:
        labels.append(1)
        
df['labels'] = labels
df.head()

,gvkey,datadate,conm,at,cogs,csho,dvp,ebit,gp,ib,...,mkvalt,sic,bankrupt,Altman_X1,Altman_X2,Altman_X3,Altman_X4,Altman_X5,Z_Score,labels
0,1240,2002-01-31,ALBERTSON'S INC,15967.000,26094.000,407.000,0.0,1787.000,11837.000,501.000,...,11701.2500,5411.0,0,0.064320,0.339074,0.111918,1.164072,2.375587,4.054972,0
1,1655,2002-01-31,ANGELICA CORP,290.865,246.466,8.608,0.0,12.856,103.597,1.629,...,101.5744,7200.0,0,0.161449,0.488845,0.044199,0.680060,1.203524,2.578078,1
2,1864,2002-01-31,REX AMERICAN RESOURCES CORP,307.329,325.912,8.163,0.0,22.236,138.591,22.554,...,240.4004,2860.0,0,0.300186,0.438319,0.072352,1.508575,1.511419,3.830851,0
3,1878,2002-01-31,AUTODESK INC,902.444,124.903,55.644,0.0,131.804,822.588,90.313,...,2281.4040,7370.0,0,0.214106,0.079664,0.146052,6.114234,1.049917,7.603973,0
4,2436,2002-01-31,CALERES INC,700.898,1044.110,17.484,0.0,54.842,711.738,0.949,...,278.6950,3140.0,0,0.321669,0.206953,0.078245,0.627350,2.505141,3.739357,0


In [6]:
from sklearn.metrics import confusion_matrix
y_true = df['bankrupt']
y_pred = df['labels']
confusion_matrix(y_true, y_pred)

array([[17376,  9262],
       [   22,   159]])

In [7]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
tn, fp, fn, tp

(17376, 9262, 22, 159)

In [8]:
from sklearn.metrics import f1_score
f1_score(y_true, y_pred)

0.03311810039575089

## Question 2

In [9]:
y = df['bankrupt'].copy()
X = df[['Altman_X1','Altman_X2','Altman_X3','Altman_X4','Altman_X5']].copy()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26819 entries, 0 to 26818
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Altman_X1  26819 non-null  float64
 1   Altman_X2  26819 non-null  float64
 2   Altman_X3  26819 non-null  float64
 3   Altman_X4  26819 non-null  float64
 4   Altman_X5  26819 non-null  float64
dtypes: float64(5)
memory usage: 1.0 MB


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30, random_state = 1)
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(random_state=1).fit(X_train, y_train)
from sklearn import tree
clf2 = tree.DecisionTreeClassifier(random_state=1).fit(X_train, y_train)
from xgboost import XGBClassifier
clf3 = XGBClassifier(random_state=1).fit(X_train, y_train) 

In [11]:
from sklearn.model_selection import cross_val_score
scores_lg = cross_val_score(clf1, X_test, y_test, cv=5)
scores_cart = cross_val_score(clf2, X_test, y_test, cv=5)
scores_xgb = cross_val_score(clf3, X_test, y_test, cv=5)
print(scores_lg)
print(scores_cart)
print(scores_xgb)

[0.99378882 0.99440646 0.99502797 0.99378496 0.99378496]
[0.99068323 0.99067744 0.98570541 0.98819142 0.98819142]
[0.99254658 0.99502797 0.99564947 0.99316346 0.99316346]


In [12]:
from sklearn.metrics import f1_score
y_pred_lg = clf1.predict(X_test)
y_pred_cart = clf2.predict(X_test)
y_pred_xgb = clf3.predict(X_test)

f1_lg = f1_score(y_test,y_pred_lg)
f1_cart = f1_score(y_test,y_pred_cart)
f1_xgb = f1_score(y_test,y_pred_xgb)

print(f1_lg)
print(f1_cart)
print(f1_xgb)

0.039999999999999994
0.11764705882352941
0.06349206349206349


## Question 3

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30, random_state = 1)

In [14]:
clf2 = tree.DecisionTreeClassifier(random_state=1).fit(X_train, y_train)

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

over = SMOTE(sampling_strategy=0.5)
steps = [('o', over)]
pipeline = Pipeline(steps=steps)
X_train, y_train= pipeline.fit_resample(X_train, y_train)

clf2 = tree.DecisionTreeClassifier(random_state=1).fit(X_train, y_train)

In [15]:
scores_cart = cross_val_score(clf2, X_test, y_test, cv=5)
print(scores_cart)

[0.99068323 0.99067744 0.98570541 0.98819142 0.98819142]


In [16]:
y_pred_cart = clf2.predict(X_test)
f1_cart = f1_score(y_test,y_pred_cart)
print(f1_cart)

0.11377245508982037


In [17]:
from imblearn.under_sampling import EditedNearestNeighbours
neigh = EditedNearestNeighbours(n_neighbors=5)
X_train, y_train = neigh.fit_resample(X_train, y_train)

In [18]:
clf2 = tree.DecisionTreeClassifier(random_state=1).fit(X_train, y_train)

In [19]:
scores_cart = cross_val_score(clf2, X_test, y_test, cv=5)
print(scores_cart)

[0.99068323 0.99067744 0.98570541 0.98819142 0.98819142]


In [20]:
y_pred_cart = clf2.predict(X_test)
f1_cart = f1_score(y_test,y_pred_cart)
print(f1_cart)

0.09111617312072892


In [21]:
y = df['bankrupt'].copy()
X = df[['Altman_X1','Altman_X2','Altman_X3','Altman_X4','Altman_X5']].copy()
class_weight = {0: 1., 1: 40}

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30, random_state = 1)
clf2 = tree.DecisionTreeClassifier(random_state=1, class_weight=class_weight).fit(X_train, y_train)

In [23]:
scores_cart = cross_val_score(clf2, X_test, y_test, cv=5)
print(scores_cart)

[0.9863354  0.99005594 0.99067744 0.98756992 0.99254195]


In [24]:
y_pred_cart = clf2.predict(X_test)
f1_cart = f1_score(y_test,y_pred_cart)
print(f1_cart)

0.10101010101010101


## Question 4

In [25]:
from keras.utils import to_categorical
y = df['bankrupt'].copy()
X = df.drop(['gvkey','datadate','conm','bankrupt','labels','Z_Score'], 1)
X['sic'] = to_categorical(X['sic'])
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26819 entries, 0 to 26818
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   at         26819 non-null  float64
 1   cogs       26819 non-null  float64
 2   csho       26819 non-null  float64
 3   dvp        26819 non-null  float64
 4   ebit       26819 non-null  float64
 5   gp         26819 non-null  float64
 6   ib         26819 non-null  float64
 7   lct        26819 non-null  float64
 8   lt         26819 non-null  float64
 9   ni         26819 non-null  float64
 10  niadj      26819 non-null  float64
 11  oiadp      26819 non-null  float64
 12  pi         26819 non-null  float64
 13  re         26819 non-null  float64
 14  revt       26819 non-null  float64
 15  sale       26819 non-null  float64
 16  wcap       26819 non-null  float64
 17  xido       26819 non-null  float64
 18  xopr       26819 non-null  float64
 19  mkvalt     26819 non-null  float64
 20  sic   

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 1)
over = SMOTE(sampling_strategy=0.5)
steps = [('o', over)]
pipeline = Pipeline(steps=steps)
X_train, y_train = pipeline.fit_resample(X_train, y_train)

In [27]:
neigh = EditedNearestNeighbours(n_neighbors=5)
X_train, y_train = neigh.fit_resample(X_train, y_train)

In [ ]:
%%time

parameters = {
    'n_estimators': [200, 400, 600],
    'max_depth': [6, 8 ,10],
    'gamma': [0.02, 0.025, 0.03],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6,0.8,1.0],
    'colsample_bytree': [0.6,0.8,1.0],
    'random_state': [1],
    'objective':['binary:logistic']
}
eval_set = [(X_train, y_train), (X_test, y_test)]
model = XGBClassifier(random_state=1)
clf = GridSearchCV(model, parameters,n_jobs=5, scoring='roc_auc')
clf.fit(X_train, y_train)

print(f'Best params: {clf.best_params_}')
print(f'Best validation score = {clf.best_score_}')

In [ ]:
%%time

model = XGBClassifier(**clf.best_params_)
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
test = pd.read_csv("test.csv")
X_test = test.drop(['id','bankrupt'], 1)
X_test['sic'] = to_categorical(X_test['sic'])

In [ ]:
y_pred = model.predict_proba(X_test)

In [ ]:
y_pred = y_pred[:,1]

In [ ]:
sample = pd.read_csv('submission_A01XXXXX.csv')
newsample = sample.drop('predict_prob', axis = 1)

In [ ]:
newsample['predict_prob'] = y_pred
newsample.to_csv('submission_A0180287Y.csv', header=True, index=False)

## Question 5

In [ ]:
plot_importance(model, max_num_features = 10)

## X2 is no longer the most important as it is not featured in the feature importance barplot. Only X5 become the most important while the others are not featured in the top 5